In [1]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
df_customers = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/customers.csv")
df_products = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/products.csv")
df_transactions = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/Transactions.csv")

In [4]:


# Lire les fichiers CSV avec le bon séparateur
df_customers = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/customers.csv", sep=';')
df_products = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/products.csv", sep=';')
df_transactions = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/Transactions.csv", sep=';')

# Afficher les premières lignes des datasets pour vérifier
print(df_customers.head())
print(df_products.head())
print(df_transactions.head())


  client_id sex  birth
0    c_4410   f   1967
1    c_7839   f   1975
2    c_1699   f   1984
3    c_5961   f   1962
4    c_5320   m   1943
  id_prod  price  categ
0  0_1421  19.99      0
1  0_1368   5.13      0
2   0_731  17.99      0
3   1_587   4.99      1
4  0_1507   3.99      0
  id_prod                        date session_id client_id
0  0_1259  2021-03-01 00:01:07.843138        s_1     c_329
1  0_1390  2021-03-01 00:02:26.047414        s_2     c_664
2  0_1352  2021-03-01 00:02:38.311413        s_3     c_580
3  0_1458  2021-03-01 00:04:54.559692        s_4    c_7912
4  0_1358  2021-03-01 00:05:18.801198        s_5    c_2033


In [5]:
# Fusionner les dataframes pour créer un seul dataframe complet
df_merged = pd.merge(df_transactions, df_customers, on='client_id', how='left')
df_merged = pd.merge(df_merged, df_products, on='id_prod', how='left')

# Afficher les premières lignes du dataframe fusionné pour vérifier
print(df_merged.head())


  id_prod                        date session_id client_id sex  birth  price  \
0  0_1259  2021-03-01 00:01:07.843138        s_1     c_329   f   1967  11.99   
1  0_1390  2021-03-01 00:02:26.047414        s_2     c_664   m   1960  19.37   
2  0_1352  2021-03-01 00:02:38.311413        s_3     c_580   m   1988   4.50   
3  0_1458  2021-03-01 00:04:54.559692        s_4    c_7912   f   1989   6.55   
4  0_1358  2021-03-01 00:05:18.801198        s_5    c_2033   f   1956  16.49   

   categ  
0      0  
1      0  
2      0  
3      0  
4      0  


In [6]:
def calculate_rfm(data):
    # Récence : Date de la dernière transaction
    data['Recency'] = (data['date_reference'] - data['last_purchase_date']).dt.days
    # Fréquence : Nombre total de transactions
    data['Frequency'] = data.groupby('customer_id')['transaction_id'].transform('count')
    # Montant : Montant total dépensé
    data['Monetary'] = data.groupby('customer_id')['amount'].transform('sum')
    return data

def create_features(data):
    # Exemple de création de nouvelles variables
    data['average_transaction_value'] = data['Monetary'] / data['Frequency']
    return data


In [7]:


def train_model(data, target):
    X = data.drop(columns=[target])
    y = data[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier()
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20]
    }
    
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    print(classification_report(y_test, y_pred))
    
    # Retourner les résultats pour analyse externe
    return X_train, X_test, y_train, y_test, y_pred, best_model
